## Reddit/Quora scraping

#### Requirements:

In [69]:
# pip install praw

In [70]:
import praw #Reddit scraper
import re #Regular expressions
import random #To select a comment
import pandas as pd

#### Reddit API use your own credentials:

In [71]:
reddit = praw.Reddit(
    client_id="LSZOduypQfuIUw",
    client_secret="4-RBX3XtMd52Q3FYRuuXb3I-9IhatQ",
    password="H4tutspa44word",
    user_agent="Tutorial",
    username="Hktuts",
)

#### Check if we're set up correctly (should return our reddit username)

In [72]:
print(reddit.user.me())

Hktuts


#### URL to the thread we want to scrape comments from:

In [76]:
url = "https://www.reddit.com/r/worldnews/comments/mo0vvc/norway_prime_minister_fined_1715_for_breaking/"
submission = reddit.submission(url=url)

#### Scraping reddit comments:

In [77]:
comments = []

submission.comments.replace_more(limit=None)

for comment in submission.comments.list():
    comments.append(comment.body)

print("Number of comments scraped from thread: " + str(len(comments)))

Number of comments scraped from thread: 1988


#### Picking a random comment from the first 50:

In [78]:
print(comments[(random.randint(1, 50))])

One rule for thee another rule for wait a second, what?


### Quora scraping
#### Scraping the question, answers and user-id:

In [79]:
#Imports

DEBUG = 1
import os
import re
import subprocess
import sys
import time
import ast
import csv
import json
import pathlib
from pathlib import Path
import random
import argparse
import sys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import dateparser

#### Function to convert answer dates dates:

In [80]:
def convertDateFormat(dateText):
	try:
		if ("Updated" in dateText):
			date = dateText[8:]
		else:
			date = dateText[9:]
		date = dateparser.parse(dateText).strftime("%Y-%m-%d")
	except:  # when updated or answered in the same week (ex: Updated Sat)
		date = dateparser.parse("7 days ago").strftime("%Y-%m-%d")
	return date

#### Function to scroll up:

In [81]:
def scrollup_alittle(self,nbtimes):

    for iii in range(0,nbtimes):
        self.execute_script("window.scrollBy(0,-200)")
        time.sleep(1)

#### Function to simulate scrolling:

In [82]:
# method for loading  quora dynamic content
def scrolldown(self,type_of_page='users'):
    last_height = self.page_source
    loop_scroll=True
    attempt = 0
    # we generate a random waiting time between 2 and 4
    waiting_scroll_time=round(random.uniform(2, 4),1)
    print('scrolling down to get all answers...')
    max_waiting_time=round(random.uniform(5, 7),1)
    # we increase waiting time when we look for questions urls	
    if type_of_page=='questions' : max_waiting_time= round(random.uniform(20, 30),1)
    # scroll down loop until page not changing
    while loop_scroll:
        self.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        if type_of_page=='answers':
            scrollup_alittle(self,2)
        new_height=self.page_source
        if new_height == last_height:
            # in case of not change, we increase the waiting time
            waiting_scroll_time= max_waiting_time
            attempt += 1
            if attempt==3:# in the third attempt we end the scrolling
                loop_scroll=False
            #print('attempt',attempt)
        else:
            attempt=0
            waiting_scroll_time=round(random.uniform(2, 4),1)
        last_height=new_height

#### Function to scrape answers:

In [83]:
def answers(urls_list,save_path):
    
    #selenium settings
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(" --window-size=1920x1080")
    
    url_index = -1
    loop_limit= 1
    
    driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')
    # give the url to scrap
    driver.get(urls_list[url_index])
    
    
    # output file containing all answers
    file_answers = open(Path(save_path) / "answers.txt", mode='a') 
    print('Starting the answers crawling...')
    
    while True:
        url_index += 1
        print('--------------------------------------------------')
        if url_index >= loop_limit:
            print('Crawling completed, answers have been saved to  :  ', save_path)
            driver.quit()
            file_answers.close()
            break
        current_line = urls_list[url_index]
        print('processing question number  : '+ str(url_index+1))
        print(current_line)
        if '/unanswered/' in str(current_line):
            print('answer is unanswered')
            continue
        question_id = current_line
        # opening Question page
        try:
            driver.get(current_line)
            time.sleep(2)
        except Exception as OpenEx:
            print('cant open the following question link : ',current_line)
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(OpenEx).__name__, OpenEx)
            print(str(OpenEx))
            continue
        try:
            nb_answers_text = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[text()[contains(.,'Answer')]]"))).text
            nb_answers=[int(s.strip('+')) for s in nb_answers_text.split() if s.strip('+').isdigit()][0]
            print('Question have :', nb_answers_text)
            
           
        except Exception as Openans: 
            print('cant get answers')
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(Openans).__name__, Openans)
            print(str(Openans))
            continue
        #nb_answers_text = browser.find_element_by_xpath("//div[@class='QuestionPageAnswerHeader']//div[@class='answer_count']").text
       
        #-------------------------------------------------------------------------------------
        
        if nb_answers > 7:
            scrolldown(driver,'answers')
        continue_reading_buttons = driver.find_elements_by_name("ChevronDown")
        time.sleep(1)
        for button in continue_reading_buttons:
            try:
                ActionChains(driver).click(button).perform()
                time.sleep(1)
            except:
                print('cant click more')
                continue
                
        print("opened a total of " + str(len(continue_reading_buttons)) + " question boxes")
        time.sleep(2)
        html_source = driver.page_source
        soup = BeautifulSoup(html_source,"html.parser")
        # get the question-id
        question_id = current_line.rsplit('/', 1)[-1]
        # find title 
        title = current_line.replace("https://www.quora.com/","")
        # find question's topics
        questions_topics = soup.findAll("div", {"class": "q-box qu-mr--tiny qu-mb--tiny"})
        questions_topics_text=[]
        for topic in questions_topics : questions_topics_text.append(topic.text.rstrip())
        # number of answers
        # not all answers are saved!
        # answers that collapsed, and those written by annonymous users are not saved
        try:
            split_html = html_source.split('q-box qu-pt--medium')
        except Exception as notexist :#mostly because question is deleted by quora
            print('question no long exists')
            print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(notexist).__name__, notexist)
            print(str(notexist))
            continue 
   
        # The underneath loop will generate len(split_html)/2 exceptions, cause answers in split_html
        # are eitheir in Odd or Pair positions, so ignore printed exceptions.
        #print('len split : ',len(split_html))
        for i in range(1, len(split_html)):
            try:
                part = split_html[i]
                part_soup = BeautifulSoup(part,"html.parser" )
                #find users names of answers authors
                try:
                    authors=part_soup.find("a", href=lambda href: href and "/profile/" in href)
                    user_id = authors['href'].rsplit('/', 1)[-1]
                    print(user_id)
                except Exception as notexist2 :#mostly because question is deleted by quora
                    print('author extract pb')
                    print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(notexist2).__name__, notexist2)
                    print(str(notexist2))
                    continue
                    
                # find answer dates
                answer_date = part_soup.find("a", string=lambda string: string and ("Answered" in string or "Updated" in string))#("a", {"class": "answer_permalink"})
                try:
                    date = answer_date.text
                    if ("Updated" in date):
                        date= date[8:]
                    else:
                        date= date[9:]
                    date=dateparser.parse(date).strftime("%Y-%m-%d")
                except: # when updated or answered in the same week (ex: Updated Sat)
                    date=dateparser.parse("7 days ago").strftime("%Y-%m-%d")
                print("Posted: " + date)
                
                # find answers text
                answer_text = part_soup.find("div", {"class": "q-relative spacing_log_answer_content puppeteer_test_answer_content"})
                print("Answer: ", answer_text.text)
                answer_text = answer_text.text                 
                
                #write answer elements to file
                s =  str(question_id.rstrip()) +'\t' + str(date) + "\t"+ user_id + "\t"+ str(questions_topics_text) + "\t" +	str(answer_text.rstrip())  + "\n"
                #print("wrting down the answer...")
                file_answers.write(s)
                print(' ')
                               
            except Exception as e1: # Most times because user is anonymous ,  continue without saving anything
                print('---------------There is an Exception-----------')
                print('Error on line {}'.format(sys.exc_info()[-1].tb_lineno), type(e1).__name__, e1)
                print(str(e1))
                o=1
                
                

#### Calling the answer function with vaccine controversial quora questions:

https://www.quora.com/Are-you-planning-on-getting-the-vaccine-for-the-coronavirus-COVID-19-when-it-comes-out
https://www.quora.com/How-safe-are-the-different-COVID-19-coronavirus-vaccinations
https://www.quora.com/What-are-the-reasons-not-to-take-the-COVID-19-vaccine


In [84]:
answers(["https://www.quora.com/What-are-the-reasons-not-to-take-the-COVID-19-vaccine",], r"C:\Users\johan\Desktop\scraping_project")

<ipython-input-83-3c47a0fffa35>:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')


Starting the answers crawling...
--------------------------------------------------
processing question number  : 1
https://www.quora.com/What-are-the-reasons-not-to-take-the-COVID-19-vaccine
Question have : 50 Answers
scrolling down to get all answers...
opened a total of 20 question boxes
Todd-Stoudt-1
Posted: 2021-01-19
Answer:  Sadly, being against taking the Covid 19 vaccine will likely get one labeled as an “antivaxer”, which I am not. One should take vaccines for illness that there are no known cure for and that we otherwise have no way to defend against and that either cause death or severe disabilities and if one is at risk from it. The overwhelming majority of those who have died of Covid 19 or suffering debilitating effects after surviving it are elderly or had one or more major health problems before getting Covid. Such persons probably should get the vaccine. Others probably do not need it. I for one do not plan on getting it. I will be very angry if schools require my kid

Posted: 2021-05-04
Answer:  If you have legit medical condition where you had gotten negative reaction to previous vaccines that didn’t revolve around COVID19, then yes, this is a legit reason that the vaccine may not be the best course unless if your doctor gives you the green light. Anything other than that is really just based on personal choice and reasoning. Nobody can ultimately force you to take the vaccine. The only time where you can be forcibly drugged without your consent is if you pose a danger to the people around you. But usually, vaccines would never generally be a factor here though.
 
Robert-Bruce-133
Posted: 2021-01-19
Answer:  Gates has programmed it with a microscopic microchip that will reassemble itself when injected. You get the shot and think there’s no microchip in there but the molecules will reassemble themselves and all your information and actions will be monitored daily by the New World Order to determine if you are worthy enough to be allowed to live in i

Posted: 2021-05-31
Answer:  Death……………………………. Brain injury , Highest number of injuries and deaths of any Injection 1 million reasons, do your homework its just your life…
 
Gervase-Warwick-Barnes
Posted: 2021-03-28
Answer:  The makers have legal indemnity, so they don't care, don't have morality, or incentive to get it right. The covid vaxes are untested, on top of that completely new, and use NANO PARTICLES, wtf would you put that inside you
 
Geoffrey-Millstone
Posted: 2021-01-19
Answer:  I can not believe that you might just attemp to be infected. The vaccine gives you a fighting chance. You are suggesting to people that there is a non medical reason to ignore a vaccine whether you believe it or not.
 
Natalie-Davies-159
Posted: 2021-01-19
Answer:  There aren’t really any reasons as to why you shouldn’t want it. Just a person’s preference.
 
Wayne-Hilborn-1
Posted: 2021-05-15
Answer:  Cruelty, idiocy, dense, unintelligent, slow, foolish, mindless, brain dead, simple-minded… ignoran

Greg-Moore-45
Posted: 2021-07-04
Answer:  Contrary to all the answers so far, it may not be related to menopause directly and in fact may actually be related to the vaccine. And no, I’m not an anti-vaxxer or anything like that. There is apparently a known correlation between vaccinations and irregular menses for a few months. About 25% of women report such an impact. I haven’t seen anything correlating to age, but I wouldn’t be surprised if there were in menopausal or peri-menopausal men.Does the COVID-19 Vaccine Affect Your Period? | On Air with Ryan Seacrest | Ryan SeacrestWe asked Dr. Danielle Jones to clarify reports and rumors the COVID vaccines could affect your cycle. Watch back the on-air convo to find out what we learned here:https://onairwithryan.iheart.com/featured/ryan-seacrest/content/2021-05-04-does-the-covid-19-vaccine-affect-your-period/Start about 3 minutes in: and 28% of women in a retrospective study showed changes. I can’t find the youtube video from Mam Doctor Jone

In [85]:
answers(["https://www.quora.com/How-safe-are-the-different-COVID-19-coronavirus-vaccinations",], r"C:\Users\johan\Desktop\scraping_project")

<ipython-input-83-3c47a0fffa35>:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')


Starting the answers crawling...
--------------------------------------------------
processing question number  : 1
https://www.quora.com/How-safe-are-the-different-COVID-19-coronavirus-vaccinations
Question have : 298 Answers
scrolling down to get all answers...
opened a total of 30 question boxes
Rachel-Anderson-166
Posted: 2021-01-26
Answer:  ORIGINAL QUESTION: Is the Covid vaccine proved safe? I am not an anti-vaxxer. I just have concerns because of how fast it was pushed through. I keep thinking about thalidomide.Thalidomide was fully and properly tested over a number of years, according to the standards of the time. It is still used to treat cancers, leprosy, and tuberculosis, and is helpful in avoiding rejection of transplant organs, stem cells and bone marrow. The problem with thalidomide is that, LIKE ALMOST EVERY OTHER DRUG IN EXISTENCE, it should NEVER BE GIVEN TO PREGNANT WOMEN. Most of the time that drug development takes is LOST TIME. It is time when NOTHING ACTUALLY HAPP

Posted: 2021-01-19
Answer:  The United States has approved two vaccines within a week for emergency medical use to prevent an outbreak of the coronavirus (COVID-19). The country with the highest death toll from the pandemic initially approved vaccines made by Pfizer and BioNtech. Then on Friday, the country approved the Moderna's Covid Vaccine.Side effects No severe long-term side effects were observed during the large-scale trial of both vaccines. However, a few short-term side effects have been observed. The Pfizer and Moderna vaccines, were not compared directly. According to trials, Moderna vaccine has a slightly higher side effect. Symptoms of fatigue, headache, and fever may appear on the first day or second day after the second dose of vaccine. This symptom is more common in people under 65 years of age. The Pfizer vaccine did not cause any side effects in humans. However, in the United States and the United Kingdom, severe allergies have been reported in some cases as a side ef

Posted: 2021-03-18
Answer:  What are your thoughts on the Coronavirus/Covid vaccine? Is it safe right now? A few preliminary notes:There is no ‘the vaccine'. There are several currently in general use. The Moderna, Pfizer and Johnson and Johnson products are being distributed and administered based on an FDA issued emergency use authorization, which is not the same as FDA approval.Any thoughts I might have on the safety of any of the three vaccines listed above are irrelevant. We have phase 3 research data supporting the release of the vaccines based on their established safety and efficacy profiles.Finally, all medical interventions have an inherent risk/benefit ratio. Currently, that ratio falls heavily on the side of vaccination as soon as possible. Very heavily. Our ongoing phase 3 test/research data indicate that the above vaccines are among the cleanest, safest and most effective vaccines ever developed. Consider, please, the following: Contemporary vaccine tech has saved more li

#### Creating a pandas dataframe with our scraped data (change to correct path):

In [86]:
df = pd.read_csv(r"C:\Users\johan\Desktop\scraping_project\answers.txt", delimiter = "	",
                 encoding='cp1252', error_bad_lines=False, header = None)
df.columns = ['Question', 'Date', 'User', 'b', 'Answer']
df = df.drop(['b'], axis=1)

b'Skipping line 71: expected 5 fields, saw 6\n'


In [87]:
df['Answer'].isnull().sum()
df = df.dropna()
df

,Question,Date,User,Answer
0,How-safe-are-the-different-COVID-19-coronaviru...,2021-01-26,Rachel-Anderson-166,ORIGINAL QUESTION: Is the Covid vaccine proved...
1,How-safe-are-the-different-COVID-19-coronaviru...,2021-01-18,Bruce-Spielbauer,"In the US, only one vaccine has been approved ..."
2,How-safe-are-the-different-COVID-19-coronaviru...,2021-04-19,Mobisoft-Infotech-1,Most of the countries are creating vaccines to...
3,How-safe-are-the-different-COVID-19-coronaviru...,2021-01-18,Frank-Hollis,Yes. My mother is being vaccinated tomorrow. I...
4,How-safe-are-the-different-COVID-19-coronaviru...,2020-12-18,C-S-Friedman,Nope. Feel free to step on out of line and han...
...,...,...,...,...
453,How-safe-are-the-different-COVID-19-coronaviru...,2021-02-15,Warren-Sponholtz,People need to have fully informed consent whe...
454,How-safe-are-the-different-COVID-19-coronaviru...,2021-02-12,Elena-Watson-28,Covid-19 Vaccine:COVID-19 vaccine is finally h...
455,How-safe-are-the-different-COVID-19-coronaviru...,2020-07-19,Bernard-Peter-Gore,"We don't “believe, in vaccines, we test them. ..."
456,How-safe-are-the-different-COVID-19-coronaviru...,2020-08-19,Patricia-Yeargin-1,"There is not yet an approved vaccine, so this ..."
